# 3.4 Property Paths

Property paths in SPARQL represent chains of the same relationship.

They can be very helpful in practice as they enable queries for paths of arbitrary length but they are computationally taxing meaning performance is often slow.

The same information can be encoded using recursive rules (see 3.3) which we can then be queried for efficiently.

## Example

The following example shows how a property path chain can be found between two nodes that aren't directly related.

Below is a social network, showing direct personal connections between people.

The goal is to find all of the people in Alice's extended network, that is to say all of the people who indirectly know Alice.

In [1]:
prop_data = """
@prefix : <https://rdfox.com/example#> .

:Alice :knows :Bob .

:Bob :knows :Carlos .

:Carlos :knows :Diana .

:Walter :knows :Wendy .

"""

In [2]:
prop_rules = """

[?person, a, :MemberOfAlicesNetwork] :-
    [:Alice, :knows, ?person].

[?person2, a, :MemberOfAlicesNetwork] :-
    [?person1, :knows, ?person2],
    [?person1, a, :MemberOfAlicesNetwork].

"""

In [4]:
import requests

# Set up the SPARQL endpoint
rdfox_server = "http://localhost:12110"

# Helper function to raise exception if the REST endpoint returns an unexpected status code
def assert_response_ok(response, message):
    if not response.ok:
        raise Exception(
            message + "\nStatus received={}\n{}".format(response.status_code, response.text))

# Clear data store
clear_response = requests.delete(
    rdfox_server + "/datastores/default/content?facts=true&axioms&rules")
assert_response_ok(clear_response, "Failed to clear data store.")

# Add data
payload = {'operation': 'add-content-update-prefixes'}
data_response = requests.patch(
    rdfox_server + "/datastores/default/content", params=payload, data=prop_data)
assert_response_ok(data_response, "Failed to add facts to data store.")

# Get rules
rules_response = requests.post(rdfox_server + "/datastores/default/content", data=prop_rules)
assert_response_ok(rules_response, "Failed to add rule.")

# Get and issue select query
with open("../queries/3_4-PropertyPathsQuery.rq", "r") as file:
    prop_query = file.read()
response = requests.get(
    rdfox_server + "/datastores/default/sparql", params={"query": prop_query})
assert_response_ok(response, "Failed to run select query.")
print('\n=== Members of Alice\'s wider Network ===')
print(response.text)


=== Members of Alice's wider Network ===
?person	?networkMember
<https://rdfox.com/example#Diana>	<https://rdfox.com/example#MemberOfAlicesNetwork>
<https://rdfox.com/example#Carlos>	<https://rdfox.com/example#MemberOfAlicesNetwork>
<https://rdfox.com/example#Bob>	<https://rdfox.com/example#MemberOfAlicesNetwork>



### Visualise the results

Open this query in the [RDFox Explorer](http://localhost:12110/console/datastores/explore?datastore=default&query=SELECT%20%3Fperson%20%3FnetworkMember%0AWHERE%20%7B%0A%20%20%20%20%3Fperson%20a%20%3FnetworkMember%20.%0A%7D).

## Exercise

Complete the rule `3_4-PropertyPathsRules.dlog` in the `rules` folder so that the query below can be used to directly find all the paths to people in Alice's network and that path required get there.

### Hits & helpful resources

In this file there are 2 sets of recursive rules, the first set iterates forward along the property path to find all network members, the second must iterate backwards to find all the people who created that specific chain.

In [38]:
prop_sparql = """

SELECT ?endPerson ?hopsAwayFromAlice ?person
WHERE {

    ?pathNode :pathTo ?endPerson;
            :hasPathLength ?pathLength;
            :pathIncludes ?person .

    ?person :hopsAwayFromAlice ?hopsAwayFromAlice.

} ORDER BY DESC(?pathLength) ASC(?endPerson) ASC(?hopsAwayFromAlice)

"""

Here is a representative sample of the data in `2_4-PropertyPathsData.ttl`.

In [22]:
sample_data = """
@prefix : <https://rdfox.com/example#> .

:Alice a :Person ;
    :knows :Jacqueline,
        :Melissa,
        :Troy .

"""

In [39]:
# Clear data store
clear_response = requests.delete(
    rdfox_server + "/datastores/default/content?facts=true&axioms&rules")
assert_response_ok(clear_response, "Failed to clear data store.")

# Get and add data
with open("../data/3_4-PropertyPathsData.ttl", "r") as file:
    data = file.read()
payload = {'operation': 'add-content-update-prefixes'}
data_response = requests.patch(
    rdfox_server + "/datastores/default/content", params=payload, data=data)
assert_response_ok(data_response, "Failed to add facts to data store.")

# Get and add rules
with open("../rules/3_4-PropertyPathsRules.dlog", "r") as rule_file:
    datalog_rule = rule_file.read()
response = requests.post(rdfox_server + "/datastores/default/content", data=datalog_rule)
assert_response_ok(response, "Failed to add rule.")

# Issue select query
response = requests.get(
    rdfox_server + "/datastores/default/sparql", params={"query": prop_sparql})
assert_response_ok(response, "Failed to run select query.")
print('\n=== How to Reach Alice\'s Network Members ===')
print(response.text)


=== Key Network Members ===
?endPerson	?hopsAwayFromAlice	?person
<https://rdfox.com/example#Clifford>	0	<https://rdfox.com/example#Alice>
<https://rdfox.com/example#Clifford>	1	<https://rdfox.com/example#Melissa>
<https://rdfox.com/example#Clifford>	2	<https://rdfox.com/example#James>
<https://rdfox.com/example#Clifford>	3	<https://rdfox.com/example#Susan>
<https://rdfox.com/example#Clifford>	4	<https://rdfox.com/example#Clifford>
<https://rdfox.com/example#Eric>	0	<https://rdfox.com/example#Alice>
<https://rdfox.com/example#Eric>	1	<https://rdfox.com/example#Melissa>
<https://rdfox.com/example#Eric>	2	<https://rdfox.com/example#James>
<https://rdfox.com/example#Eric>	3	<https://rdfox.com/example#Susan>
<https://rdfox.com/example#Eric>	4	<https://rdfox.com/example#Eric>
<https://rdfox.com/example#Glen>	0	<https://rdfox.com/example#Alice>
<https://rdfox.com/example#Glen>	1	<https://rdfox.com/example#Melissa>
<https://rdfox.com/example#Glen>	2	<https://rdfox.com/example#James>
<https:/

## Visualise the results

These power in these results is much clearer when visualised.

Open them in the RDFox Explorer [here](http://localhost:12110/console/datastores/explore?datastore=default&query=SELECT%20%3FendPerson%20%3Fperson%0AWHERE%20%7B%0A%0A%20%20%20%20%3FpathNode%20%3ApathTo%20%3FendPerson%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%3AhasPathLength%20%3FpathLength%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%3ApathIncludes%20%3Fperson%20.%0A%0A%20%20%20%20%3Fperson%20%3AhopsAwayFromAlice%20%3FhopsAwayFromAlice.%0A%0A%7D%20ORDER%20BY%20DESC%28%3FpathLength%29%20ASC%28%3FendPerson%29%20ASC%28%3FhopsAwayFromAlice%29) and **change the layout to Breadth First**.

### You should see...

=== How to Reach Alice's Network Members ===
|?endPerson|?hopsAwayFromAlice|?person|
|-----------|-------------|-------------|
|<https://rdfox.com/example#Clifford>	|0|	<https://rdfox.com/example#Alice>|
|<https://rdfox.com/example#Clifford>	|1|	<https://rdfox.com/example#Melissa>|
|<https://rdfox.com/example#Clifford>	|2|	<https://rdfox.com/example#James>|
|<https://rdfox.com/example#Clifford>	|3|	<https://rdfox.com/example#Susan>|
|<https://rdfox.com/example#Clifford>	|4|	<https://rdfox.com/example#Clifford>|
|<https://rdfox.com/example#Eric>	|0|	<https://rdfox.com/example#Alice>|
|<https://rdfox.com/example#Eric>	|1|	<https://rdfox.com/example#Melissa>|
|<https://rdfox.com/example#Eric>	|2|	<https://rdfox.com/example#James>|
|<https://rdfox.com/example#Eric>	|3|	<https://rdfox.com/example#Susan>|
|<https://rdfox.com/example#Eric>	|4|	<https://rdfox.com/example#Eric>|
|<https://rdfox.com/example#Glen>	|0|	<https://rdfox.com/example#Alice>|
|<https://rdfox.com/example#Glen>	|1|	<https://rdfox.com/example#Melissa>|
|<https://rdfox.com/example#Glen>	|2|	<https://rdfox.com/example#James>|
|<https://rdfox.com/example#Glen>	|3|	<https://rdfox.com/example#Susan>|
|<https://rdfox.com/example#Glen>	|4|	<https://rdfox.com/example#Glen>|
|<https://rdfox.com/example#Amanda>	|0|	<https://rdfox.com/example#Alice>|
|<https://rdfox.com/example#Amanda>	|1|	<https://rdfox.com/example#Melissa>|
|<https://rdfox.com/example#Amanda>	|2|	<https://rdfox.com/example#James>|
|<https://rdfox.com/example#Amanda>	|3|	<https://rdfox.com/example#Amanda>|
|<https://rdfox.com/example#Bobby>	|0|	<https://rdfox.com/example#Alice>|
|<https://rdfox.com/example#Bobby>	|1|	<https://rdfox.com/example#Troy>|
|<https://rdfox.com/example#Bobby>	|2|	<https://rdfox.com/example#Terri>|
|...|||
|<https://rdfox.com/example#Melissa>	|1|	<https://rdfox.com/example#Melissa>|
|<https://rdfox.com/example#Troy>	|0|	<https://rdfox.com/example#Alice>|
|<https://rdfox.com/example#Troy>	|1|	<https://rdfox.com/example#Troy>|